In [1]:
import urllib.request 
from urllib.parse import quote 
from bs4 import BeautifulSoup
import json
import re
import time

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
#Source: https://codezup.com/web-scraping-word-meaning-dictionary-python-beautifulsoup/

In [4]:
#getting the already scraped dict
import json
with open('/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/vdict VI-FR ver 2.json') as json_file:
     currDict = json.load(json_file)

In [5]:
#reading the word list Andras gave:
with open('/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/raw5') as infile:
  wordsData = infile.readlines()

In [6]:
#generate a list of words from the word list Andras gave
def generateWordList(): 
  temp = []
  for word in wordsData:
    word = word.strip()
    word = word.split(" ")[0]
    temp.append(word)
  return temp

In [7]:
wordList = generateWordList()
len(wordList)

31895

In [8]:
#break the word list into batches of 
batches = [wordList[i:i + 10000] for i in range(0, len(wordList), 10000)] 

In [9]:
for batch in batches:
  print(len(batch))

10000
10000
10000
1895


In [10]:
print(len(batches))

4


In [11]:
#The link format for vdict Viet-Fran is: https://vdict.com/m%E1%BB%99t,4,0,0.html

In [12]:
#Converting IRI to ASCII:
#https://stackoverflow.com/questions/4389572/how-to-fetch-a-non-ascii-url-with-python-urlopen

In [13]:
def findTranslation(word):
    print("Finding translation for " + word)
    url = "https://vdict.com/" + quote(word) + ",4,0,0.html" #resolving the IRI issue
    
    try: #make sure the link is working
        source = urllib.request.urlopen(url)
    except:
        print("Link broken for " + word)
        return "N/A"
    
    soup = BeautifulSoup(source, 'lxml')

    #make sure the word is in the dictionary
    if (soup.find("div", id = 'result-contents') is None):
        print("Not in the dictionary " + word)
        return "N/A"

    translations = {}

    #Getting the first translation
    curTag = soup.find("ul", class_= "list1")

    if (curTag.find('b') is None): #empty translation page
        print("Empty translation page " + word)
        return "N/A"

    #storing the number of translations a word has
    numTrans = 1

    #continue while we still have more translations
    while(curTag != '\n'): 
        temp = {} #temp dict to store each translation of a word

        #getting the raw translation
        if (curTag.find('b').string is not None): #empty translation cell
            temp['translation'] = re.split('[,;]', curTag.find('b').string)

            #retrieving the examples of each of the translations
            try:
                examples = curTag.findAll("ul", class_= "list2")
                temp2 = {} #another dict to store each examples of a translation

                numExamples = len(list(examples))
                temp2['nums'] = numExamples

                for j in range(0, numExamples):
                    temp2['context ' + str(j+1)] = examples[j].find(class_ = "example-original").string

                    #retrieving the usage of each examples
                    try: 
                        temp2['usage ' + str(j+1)] = str(examples[j].find("li").contents[-1])
                    except:
                        temp2['usage ' + str(j+1)] = "N/A"

                temp['examples'] = temp2
            except:
                temp['examples'] = "N/A"

            translations[numTrans] = temp #adding the translation to the dictionary

            numTrans += 1
        #go to next tag  
        curTag = curTag.next_sibling
    
    translations["nums"] = numTrans - 1
        
    return translations

In [14]:
url = "https://vdict.com/" + quote("có") + ",4,0,0.html" #resolving the IRI issue

try:
    source = urllib.request.urlopen(url)
except:
    print( "Link broken")
soup = BeautifulSoup(source, 'lxml')

In [15]:
soup.find("ul", class_= "list1").next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling

'\n'

In [16]:
containers = soup.findAll("ul", class_= "list1")
containers

[<ul class="list1"><li><b>avoir; posséder</b><ul class="list2"><li><span class="example-original">Tôi có nói câu đó </span><br/>j'ai dit cette phrase</li></ul><ul class="list2"><li><span class="example-original">Có tiền </span><br/>avoir de l'argent</li></ul><ul class="list2"><li><span class="example-original">Có một cái nhà </span><br/>posséder une maison</li></ul></li></ul>,
 <ul class="list1"><li><b>exister; se trouver</b><ul class="list2"><li><span class="example-original">Cái tục đó còn đó </span><br/>cet usage existe encore</li></ul><ul class="list2"><li><span class="example-original">Loài chim đó không có ở Việt Nam </span><br/>cette espèce d'oiseau n'existe pas au Viet-Nam</li></ul><ul class="list2"><li><span class="example-original">Tên anh không có trong danh sách </span><br/>ton nom ne se trouve pas sur la liste</li></ul></li></ul>,
 <ul class="list1"><li><b>disposer de</b><ul class="list2"><li><span class="example-original">ông ấy có một cái xe </span><br/>il dispose d'une 

In [17]:
meanings = [item.find('b').string for item in containers[0:4]] #word translation
meanings

['avoir; posséder', 'exister; se trouver', 'disposer de', 'être présent']

In [18]:
examples = containers[0].findAll("ul", class_= "list2")
examples[1].find("li").contents[-1]

"avoir de l'argent"

In [19]:
from google.colab import files
start_time = time.time()

index = 3
batch = batches[index] 

#for (index, batch) in enumerate(batches):
dictionary = {}

#making the dictionary for each batch
for word in batch:

    #if the word is not in already in the dict, find the translation
    if word not in currDict:
      dictionary[word] = findTranslation(word)
      time.sleep(1) #wait for 1 sec between each query

Finding translation for đặtvé
Not in the dictionary đặtvé
Finding translation for đăkrông
Not in the dictionary đăkrông
Finding translation for ía
Not in the dictionary ía
Finding translation for zuhair
Not in the dictionary zuhair
Finding translation for zhan
Not in the dictionary zhan
Finding translation for yukawa
Not in the dictionary yukawa
Finding translation for yueh
Not in the dictionary yueh
Finding translation for yoshiro
Not in the dictionary yoshiro
Finding translation for ybm
Not in the dictionary ybm
Finding translation for xxxxxxx
Not in the dictionary xxxxxxx
Finding translation for wolfberry
Not in the dictionary wolfberry
Finding translation for webbạn
Not in the dictionary webbạn
Finding translation for vânthể
Not in the dictionary vânthể
Finding translation for vvvv
Not in the dictionary vvvv
Finding translation for vppa
Not in the dictionary vppa
Finding translation for volfram
Not in the dictionary volfram
Finding translation for volachith
Not in the dictionary vo

In [20]:
#saving the batch
with open('/content/gdrive/My Drive/Colab Notebooks/Dictionary Project/vdict VI-FR new ver batch {}.json'.format(index), 'w') as outfile:
  json.dump(dictionary, outfile)
  
print("--- %s seconds ---" % (time.time() - start_time))

--- 2907.8605592250824 seconds ---


In [21]:
len(dictionary.items())

1880

In [22]:
findTranslation("denys")

Finding translation for denys
Not in the dictionary denys


'N/A'